In [1]:
import json
import time
import random
from collections import defaultdict
from tqdm import tqdm
import google.generativeai as genai

genai.configure(api_key="<paste_your_gemini_api_key_here>")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def build_prompt(question, shots=[]):

    prompt_instructions = (
        '''You are a SQL generation assistant for the ATIS flight database. Given a natural language question, generate a corresponding SQL query following these strict rules:

    ### SQL Generation Rules
    1. Always use SELECT DISTINCT.
    2. Do NOT use JOIN. Use comma-separated tables in the FROM clause, joined using WHERE conditions with parentheses.
    3. Alias every table in the format: [TABLE] AS [TABLEaliasN], where N starts from 0 and increases for each table instance.
    4. Always qualify fields with their table aliases (e.g., FLIGHTalias0.FLIGHT_ID).
    5. Do NOT use SELECT *, GROUP BY, HAVING, LIMIT, or OFFSET.
    6. Output the SQL as a single-line string — no line breaks or indentation.
    7. Only use tables and fields from the schema listed below.
    8. Include WHERE clause conditions for the tables with [TABLEaliasN] (where N > 0) in parentheses with the joining clauses.

    ---

    ### Additional SQL Features from Dataset
    • Subqueries for MIN, MAX, or nested filtering
    • IN (SELECT ...) for mappings (e.g., DAYS_CODE)
    • OR logic for alternatives
    • Indirect joins (e.g., CITY → AIRPORT_SERVICE → FLIGHT)
    • Repeated use of same tables with increasing aliases
    • Use of NOT, >=, < for filtering
    • Outputs are mostly IDs or codes
    • Always end SQL with a semicolon
    • Always have a space before and after parentheses

    ---

    ### City-Based Query Rules
    • Use CITY.CITY_NAME joined via AIRPORT_SERVICE and FLIGHT
    • Use FLIGHT.FROM_AIRPORT and FLIGHT.TO_AIRPORT for route matching
    • Ensure CITYalias0 refers to origin, CITYalias1 refers to destination
    • Only include CITYalias1-based filters and FLIGHTalias0.TO_AIRPORT in the main parenthesis**
    • Place CITYalias0-based filters and FLIGHTalias0.FROM_AIRPORT outside the parenthesis**
    • Put TO_AIRPORT-based filtering (e.g., DEPARTURE_TIME) in CITYalias1 clause

    ---

    ### Fare and Round Trip Rules
    • Use FARE, FLIGHT_FARE, and FLIGHT when the question asks about prices, tickets, costs, round trips, or arrangements
    • For round trip queries, always include:
        FAREalias0.ROUND_TRIP_COST IS NOT NULL AND FLIGHT_FAREalias0.FARE_ID = FAREalias0.FARE_ID AND FLIGHTalias0.FLIGHT_ID = FLIGHT_FAREalias0.FLIGHT_ID
    • For one way flights, always include:
        FAREalias0.ROUND_TRIP_REQUIRED = "NO" AND FLIGHT_FAREalias0.FARE_ID = FAREalias0.FARE_ID AND FLIGHTalias0.FLIGHT_ID = FLIGHT_FAREalias0.FLIGHT_ID
    • Do NOT use `ONE_DIRECTION_COST IS NOT NULL` to infer one-way flights because it is not correct.

    ---

    ### Date vs Time Handling
    • Use DATE_DAY and DAYS only when the question is about days of the week or calendar dates (e.g., "on Monday", "May 13th")
    • Do NOT use DATE_DAY and DAYS when the question is only about departure or arrival times (e.g., "before 1000")

    ---

    ### Time Filter Rules
    • If the question mentions:
        - “before <time>”: use FLIGHTalias0.DEPARTURE_TIME < [time]
        - “after <time>”: use FLIGHTalias0.DEPARTURE_TIME > [time]
        - “between <t1> and <t2>”: use FLIGHTalias0.ARRIVAL_TIME >= [t1] AND FLIGHTalias0.ARRIVAL_TIME <= [t2]
        - “around <time>” or “about <time>”: use a window of ±30 minutes from the time:
            FLIGHTalias0.ARRIVAL_TIME >= [time - 100] AND FLIGHTalias0.ARRIVAL_TIME <= [time + 100]
    • Time filters should always go in the CITYalias1 (destination) clause.
    • Use FLIGHTalias0.ARRIVAL_TIME if arrival is mentioned; otherwise, use FLIGHTalias0.DEPARTURE_TIME.

    ---

    ### Output Field Rule
    • Unless the question explicitly asks for cost, times, or other fields, always output:
        SELECT DISTINCT FLIGHTalias0.FLIGHT_ID

    ---

    ### Full Table Schema
    • AIRLINE(AIRLINE_CODE, AIRLINE_NAME)
    • AIRPORT(AIRPORT_CODE, AIRPORT_LOCATION, AIRPORT_NAME, STATE_CODE, TIME_ZONE_CODE)
    • AIRPORT_SERVICE(AIRPORT_CODE, CITY_CODE)
    • CITY(CITY_CODE, CITY_NAME, STATE_CODE, COUNTRY_NAME, COUNTRY_ABBREVIATION, TIME_ZONE_CODE)
    • CLASS(CLASS_CODE, CLASS_DESCRIPTION)
    • CONNECTION(CONNECTION_ID, FLIGHT_ID, LEG_NUMBER, FROM_AIRPORT, TO_AIRPORT, MIN_CONNECTION_TIME)
    • DAYS(DAYS_CODE, DAY_NAME)
    • DATE_DAY(DATE_CODE, DAY_NUMBER, MONTH_NUMBER, YEAR, DAY_NAME)
    • EQUIPMENT(EQUIPMENT_CODE, EQUIPMENT_DESCRIPTION)
    • FARE(FARE_ID, FARE_BASIS_CODE, FARE_AIRLINE, FROM_AIRPORT, TO_AIRPORT, RESTRICTION_CODE, ROUND_TRIP_REQUIRED, ONE_DIRECTION_COST, ROUND_TRIP_COST)
    • FARE_BASIS(FARE_BASIS_CODE, FARE_DESCRIPTION)
    • FLIGHT(FLIGHT_ID, AIRLINE_CODE, FLIGHT_NUMBER, EQUIPMENT_CODE, FROM_AIRPORT, TO_AIRPORT, DEPARTURE_TIME, ARRIVAL_TIME, FLIGHT_TIME, MEAL_CODE, FLIGHT_DAYS, STOP_INFO, CODE_SHARE)
    • FLIGHT_LEG(FLIGHT_LEG_ID, FLIGHT_ID, LEG_NUMBER, FROM_AIRPORT, TO_AIRPORT, DEPARTURE_TIME, ARRIVAL_TIME)
    • FLIGHT_LEG_STOP(FLIGHT_LEG_ID, LEG_NUMBER, STOP_NUMBER, STOP_AIRPORT, ARRIVAL_TIME, DEPARTURE_TIME)
    • FLIGHT_STOP(FLIGHT_ID, STOP_NUMBER, STOP_AIRPORT, ARRIVAL_TIME, DEPARTURE_TIME)
    • GROUND_SERVICE(GROUND_SERVICE_ID, AIRPORT_CODE, GROUND_TRANSPORT_TYPE, SERVICE_NAME, CITY_CODE, TIME)
    • MEAL(MEAL_CODE, MEAL_DESCRIPTION)
    • RESTRICTION(RESTRICTION_CODE, RESTRICTION_DESCRIPTION)
    • STATE(STATE_CODE, STATE_NAME)
    • TIME_ZONE(TIME_ZONE_CODE, TIME_ZONE_NAME)

    ---

    ### Sample Field Values (Using Field Names)
    • CITY.CITY_NAME = "BOSTON", "NEW YORK", "CHICAGO"
    • AIRPORT.AIRPORT_CODE = "JFK", "SFO", "LAX"
    • AIRLINE.AIRLINE_CODE = "AA", "DL", "UA"
    • FLIGHT.DEPARTURE_TIME = 600, 1200, 1800
    • FLIGHT.FLIGHT_NUMBER = "343", "122"
    • DATE_DAY.DAY_NAME = "FRIDAY", "MONDAY"
    • DATE_DAY.MONTH_NUMBER = 5
    • DATE_DAY.YEAR = 1991
    • FARE.ONE_DIRECTION_COST = 750
    • FARE.ROUND_TRIP_COST = 1500
    • FARE.ROUND_TRIP_REQUIRED = "NO"
    • FARE_BASIS.CLASS_TYPE = "ECONOMY", "FIRST"
    • CLASS_OF_SERVICE.BOOKING_CLASS = "Y", "F", "B"
    '''
    )

    formatted_shots = []
    for ex in shots:
        formatted_shots.append(f"Q: {ex['question']} => {ex['sql']}")

    return prompt_instructions + "\n".join(formatted_shots) + f"\nQ: {question} =>"

In [6]:
def generate_sql(question, shots=[], retries=20, base_wait=2):

    prompt = build_prompt(question, shots)
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel("gemini-1.5-pro")
            response = model.generate_content(prompt)
            return response.text.strip()

        except Exception as e:
            wait = base_wait * (attempt + 1)
            print(f"Error: {e}. Waiting {wait}s...")
            time.sleep(wait)
    raise RuntimeError("Runtime Error: Failed to generate after multiple retries.")

def load_shot_examples(examples, shots):

    shot_examples = []
    for ex in examples.get("train", []):
        q = ex.get("question", "").strip()
        sql = ex.get("final_sql", "").strip()
        if q and sql:
            shot_examples.append({"question": q, "sql": sql})

    for ex in examples.get("dev", []):
        q = ex.get("question", "").strip()
        sql = ex.get("final_sql", "").strip()
        if q and sql:
            shot_examples.append({"question": q, "sql": sql})
    # random.seed(5046)
    random.shuffle(shot_examples)
    return shot_examples[:shots]

def get_eval_examples(examples, split):

    eval_examples = []
    for ex in examples.get(split, []):
        q = ex.get("question", "").strip()
        sql = ex.get("final_sql", "").strip()
        if q and sql:
            eval_examples.append({"question": q, "final_sql": sql})
    return eval_examples

def evaluate(eval_examples, shots):

    correct = 0
    for example in tqdm(eval_examples):
        question = example["question"]
        final = example["final_sql"]
        predicted = generate_sql(question, shots)
        print("\nQuestion:", question)
        print("Predicted SQL:", predicted)
        print("Correct SQL:", final)
        if predicted == final:
            print("CORRECT!!!\n")
            correct += 1
        else:
            print("INCORRECT\n")

    print(f"Accuracy: {correct}/{len(eval_examples)} = {(correct/len(eval_examples)):.5f}")

In [4]:
with open("atis_query_split.json") as dataset:
    examples = json.load(dataset)

In [ ]:
print("Evaluating the LLM Gemini 1.5 Pro on ATIS by \"query-split\" and 0-shot training (no examples)...")
evaluate(get_eval_examples(examples, split="test"), load_shot_examples(examples, 0))

In [7]:
print("Evaluating the LLM Gemini 1.5 Pro on ATIS by \"query-split\" and 5-shot training (few examples)...")
evaluate(get_eval_examples(examples, split="test"), load_shot_examples(examples, 5))

Evaluating LLM Gemini 1.5 Pro on ATIS by "query-split" and 5-shot training (few examples)...


  0%|          | 1/347 [00:02<15:56,  2.76s/it]


Question: do you have an 819 flight from DENVER to SAN FRANCISCO
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "SAN FRANCISCO" AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "DENVER" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE AND FLIGHTalias0.FLIGHT_NUMBER = 819 ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "SAN FRANCISCO" AND FLIGHTalias0.DEPARTURE_TIME = 81

  1%|          | 2/347 [00:05<16:20,  2.84s/it]


Question: i would like a flight from KANSAS CITY to ST. PAUL departing at 1600
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "ST. PAUL" AND FLIGHTalias0.DEPARTURE_TIME = 1600 AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "KANSAS CITY" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "ST. PAUL" AND FLIGHTalias0.DEPARTUR

...


Question: what are the nonstop flights between SAN JOSE and HOUSTON TEXAS
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "HOUSTON" AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "SAN JOSE" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE AND FLIGHTalias0.STOPS = 0 ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE ( ( FLIGHTalias0.FROM_AIRPORT IN ( SELECT AIRPORT_SERVICEalias0.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 WHERE AIRPORT_SERVICEalias0.CITY_CODE IN ( SELECT CITYalias0.CITY_CODE FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "SAN JOSE" ) ) AND FLI

100%|█████████▉| 346/347 [20:36<00:03,  3.34s/it]


Question: find AA flight from NEWARK to NASHVILLE around 630pm
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , AIRLINE AS AIRLINEalias0 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "NASHVILLE" AND FLIGHTalias0.ARRIVAL_TIME >= 1800 AND FLIGHTalias0.ARRIVAL_TIME <= 1900 AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "NEWARK" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE AND AIRLINEalias0.AIRLINE_CODE = "AA" AND FLIGHTalias0.AIRLINE_CODE = AIRLINEalias0.AIRLINE_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE ( ( ( ( FLIGHTalias0.DEPARTURE_TIME <= 1900 AND FLIGHTalias0.DEPARTURE_TIME >= 1800 ) AND FLIGHTalias0.TO_A

100%|██████████| 347/347 [20:41<00:00,  3.58s/it]


Question: please find a flight round trip from LOS ANGELES to TACOMA WASHINGTON with a stopover in SAN FRANCISCO not exceeding the price of 300 dollars for 6 10 1993
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias2 , CITY AS CITYalias0 , CITY AS CITYalias1 , CITY AS CITYalias2 , DATE_DAY AS DATE_DAYalias0 , FARE AS FAREalias0 , FLIGHT AS FLIGHTalias0 , FLIGHT_FARE AS FLIGHT_FAREalias0 , FLIGHT_STOP AS FLIGHT_STOPalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "TACOMA" AND CITYalias1.STATE_CODE = "WA" AND CITYalias2.CITY_CODE = AIRPORT_SERVICEalias2.CITY_CODE AND CITYalias2.CITY_NAME = "SAN FRANCISCO" AND DATE_DAYalias0.DAY_NUMBER = 10 AND DATE_DAYalias0.MONTH_NUMBER = 6 AND DATE_DAYalias0.YEAR = 1993 AND FAREalias0.ROUND_TRIP_COST <= 300 AND FAREalias0.ROUND_TRIP_COST IS NOT NULL AND FLIGHT_FAREalias0.

In [5]:
print("Evaluating the LLM Gemini 1.5 Pro on ATIS by \"query-split\" and 40-shot training (many examples)...")
evaluate(get_eval_examples(examples, split="test"), load_shot_examples(examples, 40))

Evaluating LLM Gemini 1.5 Pro on ATIS by "query-split" and 40-shot training (many examples)...


  0%|          | 1/347 [00:03<22:50,  3.96s/it]


Question: do you have an 819 flight from DENVER to SAN FRANCISCO
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , DATE_DAY AS DATE_DAYalias0 , DAYS AS DAYSalias0 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "SAN FRANCISCO" AND DATE_DAYalias0.DAY_NUMBER = 19 AND DATE_DAYalias0.MONTH_NUMBER = 8 AND DATE_DAYalias0.YEAR = 1991 AND DAYSalias0.DAY_NAME = DATE_DAYalias0.DAY_NAME AND FLIGHTalias0.FLIGHT_DAYS = DAYSalias0.DAYS_CODE AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "DENVER" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVIC

  1%|          | 2/347 [00:07<21:03,  3.66s/it]


Question: i would like a flight from KANSAS CITY to ST. PAUL departing at 1600
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "ST. PAUL" AND FLIGHTalias0.DEPARTURE_TIME = 1600 AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "KANSAS CITY" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "ST. PAUL" AND FLIGHTalias0.DEPARTUR

...


Question: what are the nonstop flights between SAN JOSE and HOUSTON TEXAS
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "SAN JOSE" AND CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "HOUSTON" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE AND FLIGHTalias0.STOP_INFO = "0" ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE ( ( FLIGHTalias0.FROM_AIRPORT IN ( SELECT AIRPORT_SERVICEalias0.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 WHERE AIRPORT_SERVICEalias0.CITY_CODE IN ( SELECT CITYalias0.CITY_CODE FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "SAN JOSE" ) ) AND F

100%|█████████▉| 346/347 [34:41<00:03,  3.63s/it]


Question: find AA flight from NEWARK to NASHVILLE around 630pm
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "NASHVILLE" AND FLIGHTalias0.ARRIVAL_TIME >= 1800 AND FLIGHTalias0.ARRIVAL_TIME <= 1900 AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "NEWARK" AND FLIGHTalias0.AIRLINE_CODE = "AA" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE ( ( ( ( FLIGHTalias0.DEPARTURE_TIME <= 1900 AND FLIGHTalias0.DEPARTURE_TIME >= 1800 ) AND FLIGHTalias0.TO_AIRPORT IN ( SELECT AIRPORT_SERVICEalias1.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_S

100%|██████████| 347/347 [34:48<00:00,  6.02s/it]


Question: please find a flight round trip from LOS ANGELES to TACOMA WASHINGTON with a stopover in SAN FRANCISCO not exceeding the price of 300 dollars for 6 10 1993
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias2 , CITY AS CITYalias0 , CITY AS CITYalias1 , CITY AS CITYalias2 , DATE_DAY AS DATE_DAYalias0 , DAYS AS DAYSalias0 , FARE AS FAREalias0 , FLIGHT AS FLIGHTalias0 , FLIGHT_FARE AS FLIGHT_FAREalias0 , FLIGHT_STOP AS FLIGHT_STOPalias0 WHERE ( ( ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "TACOMA" AND CITYalias2.CITY_CODE = AIRPORT_SERVICEalias2.CITY_CODE AND CITYalias2.CITY_NAME = "SAN FRANCISCO" AND DATE_DAYalias0.DAY_NUMBER = 10 AND DATE_DAYalias0.MONTH_NUMBER = 6 AND DATE_DAYalias0.YEAR = 1993 AND DAYSalias0.DAY_NAME = DATE_DAYalias0.DAY_NAME AND FAREalias0.ROUND_TRIP_COST <= 300 AND FLIGHT_FAREalias0.F